The instructions provided to us are vague so I could use Suricata to employ the rules by iteslf, but we believe that would be agains the point since you the insrtucors would have to follow our exact Suricata set up. Addtionally Suricata does not have a python api the only way to use it with pyhton is to read the logs and that is what we did, but we undertsand that it is not nessary but we need to prove our understanding of the concept. 

Before we can even start we need to set up suricata correctly please do the following in config file //etc/suricata/suricata.yaml or euqivalent you need to change two lines in af-packet: <correct interface> and rules-files: you need to add a - custom.rules
you will also need to create that file in //var/lib/suricata/rules and you will add the line bellow

In [8]:
alert tcp $HOME_NET any -> $HOME_NET any (msg:"Possible Syn Scan"; flags:S; threshold:type both, track by_src, count 10, seconds 10; sid:1000002; rev:1;)


SyntaxError: invalid syntax (936203676.py, line 1)

finally to make sure suircata is running update it, then run this command

In [ ]:
sudo suricata -i virbr0 -c //etc/suricata/suricata.yaml

replace virbr0 with what every iterface that you see fit, fianlly restart and suricata is ready

In [ ]:
import json
import subprocess

def ensure_nftables_table_exists():
    """
    Ensures that the nftables 'filter' table and 'input' chain exist.
    Uses the 'ip' family instead of 'inet' to avoid conflicts with existing configurations.
    """
    try:
        # Check if the 'filter' table exists under the 'ip' family
        result = subprocess.run(['sudo', 'nft', 'list tables'], capture_output=True, text=True, check=True)
        if 'filter' not in result.stdout:
            print("Creating 'filter' table in 'ip' family...")
            subprocess.run(['sudo', 'nft', 'add', 'table', 'ip', 'filter'], check=True)

        # Check if the 'input' chain exists in the 'filter' table
        result = subprocess.run(['sudo', 'nft', 'list ruleset'], capture_output=True, text=True, check=True)
        if 'chain input' not in result.stdout:
            print("Creating 'input' chain in 'filter' table...")
            subprocess.run([
                'sudo', 'nft', 'add', 'chain', 'ip', 'filter', 'input',
                '{ type filter hook input priority 0; }'
            ], check=True)

    except subprocess.CalledProcessError as e:
        print(f"Error ensuring nftables table: {e}")
        if e.stderr:
            print(f"stderr: {e.stderr.decode()}")
        else:
            print("stderr: None")

def block_ip_with_nft(ip):
    try:
        print(f"Blocking IP: {ip} using nftables")

        # Ensure the correct nftables table and chain exist before adding the rule
        ensure_nftables_table_exists()

        # Add rule to block the IP
        result = subprocess.run([
            'sudo', 'nft', 'add', 'rule', 'ip', 'filter', 'input', 'ip', 'saddr', ip, 'drop'
        ], check=True, capture_output=True)

        print(f"IP {ip} blocked successfully.")
        print(f"nftables result: {result.stdout.decode()}")

    except subprocess.CalledProcessError as e:
        print(f"Error applying nftables rule: {e}")
        if e.stderr:
            print(f"stderr: {e.stderr.decode()}")
        else:
            print("stderr: None")

def read_json_file(file_path, allowed_ports):
    ips_to_block = []  # List of IPs to block

    try:
        with open(file_path, 'r') as file:
            for line in file:
                try:
                    entry = json.loads(line.strip())
                    if entry.get("event_type") == "alert" and entry.get("alert", {}).get("signature") == "Possible Syn Scan":
                        src_ip = entry.get("src_ip")
                        dst_port = entry.get("dest_port")
                        if dst_port not in allowed_ports:
                            if src_ip not in ips_to_block:  # Avoid duplicate IPs
                                ips_to_block.append(src_ip)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")

    except Exception as e:
        print(f"Error reading JSON file: {e}")

    # Block IPs that are found in the list
    for ip in ips_to_block:
        block_ip_with_nft(ip)

def main():
    # Specify the path to your Suricata eve.json log file
    eve_log_path = '/var/log/suricata/eve.json'
    allowed_ports = ["22","53","80"]
    # Read the log and block IPs
    read_json_file(eve_log_path, allowed_ports)



In [9]:
main()

Blocking IP: 192.168.124.139 using nftables
IP 192.168.124.139 blocked successfully.
nftables result: 


Hey this might glitch please export to indivdual python file and run with sudo to make it work, if you run main and nothing happens the notebook is glitched but the code is correct